# 多变量KNN模型：预测一个年龄为30，烟龄为12，没有Cancer的人，是否有DHPV

In [21]:
# 导入所用的读取数据的包，用于对数据进行预处理： 标准化的函数StandardScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 读取所需要的文件
data = pd.read_csv('./data/risk.csv')



# 对属性列，进行转换类型
# 找出所有的属性名，并转换成列表
cols = list(data.columns)
# 将所有的列都转换成数值型，将转换过程中出错的，填入Nan,然后填入0
data = data[cols].apply(pd.to_numeric,errors = 'coerce').fillna('0.0')
# 最后将所有的列转换为float型
data = pd.DataFrame(data,dtype = 'float')
# 查看属性列的类型: 全为float型
data.dtypes




# 将数据中的属性列重新命名: columns = {'Smokes':'Smoke','Smokes (years)':'Smokeyear'}
#    ,inplace = True: 一定要指定为True，否则就相当于没有任何作用               
data.rename(columns = {'Smokes':'Smoke','Smokes (years)':'Smokeyear',
                              'Dx:Cancer':'DC','Dx:HPV':'DHPV'},inplace = True)

# 选取特征值的属性名
feature = ['Age','Smoke','Smokeyear','DC','DHPV']
# 从数据集中选出特征值的属性列，并且赋值给新的变量
data = data[feature]



data['Smokeyear'] = data.Smokeyear.astype(float)


# 将该数据转换成标准化数据
data[feature] = StandardScaler().fit_transform(data[feature])
print(data.shape)
data.head()


(858, 5)


,Age,Smoke,Smokeyear,DC,DHPV
0,-1.038563,-0.409081,-0.295999,-0.146385,-0.146385
1,-1.391796,-0.409081,-0.295999,-0.146385,-0.146385
2,0.845342,-0.409081,-0.295999,-0.146385,-0.146385
3,2.964736,2.444506,8.821219,6.831301,6.831301
4,2.258271,-0.409081,-0.295999,-0.146385,-0.146385


# 此时就可以得到一个经过标准化与处理的数据了

In [22]:
# 查看属性列的类型
# data.dtypes

In [23]:
# 划分训练集和测试集
norm_train_df = data.copy().iloc[:636]
norm_test_df = data.copy().iloc[636:]

多变量的距离的计算

In [24]:
# scipy 中关于统计的包，导入欧氏距离
from scipy.spatial import distance

# 拿到三个样本
first_data = data.iloc[0][['Age','Smokeyear','DC']]
fifth_data = data.iloc[20][['Age','Smokeyear','DC']]

# 直接调入distance 中的欧氏距离，将两个样本传进去，distance.euclidean()
first_fifth_distance = distance.euclidean(first_data,fifth_data)
first_fifth_distance

2.94360254018141

# 多变量KNN模型

In [25]:
# 评估：  多变量预测dhpv模型
def predict_dhpv_multivariate(new_data_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_data_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn8 = temp_df.DHPV.iloc[:5]
    predict_dhpv = knn8.mean()
    return(predict_dhpv)

cols = ['Age','Smokeyear','DC']
norm_test_df['predict_dhpv'] = norm_test_df[cols].apply(predict_dhpv_multivariate,feature_columns = cols,axis = 1)
norm_test_df['squared_error'] = (norm_test_df['predict_dhpv'] - norm_test_df['DHPV'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

0.4956144001475678


# 使用Sklearn 来完成KNN 

In [33]:
from sklearn.neighbors import KNeighborsRegressor

cols = ['Age','Smokeyear','DC']
# 实例化一个可能knn 的对象
# 默认值为n_neighbors = 5,可以通过调节值的大小,来找到误差值的最小值
knn = KNeighborsRegressor(n_neighbors = 5)

# 利用Fit() 函数，进行模型的训练，（） 函数中传进来训练集的x(norm_train_df[cols],
                          #  y (norm_train_df['DHPV'])
knn.fit(norm_train_df[cols],norm_train_df['DHPV'])

# 用测试集预测测试集的结果
three_feature_predictions = knn.predict(norm_test_df[cols])

In [34]:
from sklearn.metrics import mean_squared_error

three_feature_mse = mean_squared_error(norm_test_df['DHPV'],three_feature_predictions)
three_feature_rmse = three_feature_mse ** (1/2)
print(three_feature_rmse)

0.4956144001475678


加入更多的特征值

In [37]:
knn = KNeighborsRegressor()

cols = ['Age','Smoke','Smokeyear','DC']
knn.fit(norm_train_df[cols],norm_train_df['DHPV'])

# 用测试集预测测试集的结果
four_feature_predictions = knn.predict(norm_test_df[cols])
four_feature_mse = mean_squared_error(norm_test_df['DHPV'],four_feature_predictions)
four_feature_rmse = four_feature_mse ** (1/2)
print(four_feature_rmse)

0.4956144001475678


# 参数更多的话，应该会使模型进行调优，选择更多的特征，
K近邻可以做分类做回归；

# 回归问题
一个例子：
- 数据：工资和年龄（2 个特征）
- 目标：预测银行会贷给我多少钱（标签）
- 考虑：工资和年龄都会影响最终银行贷款的结果那么各自有多大的影响呢？（参数）
- 找到最合适的一条线，（想像一个高维）来最好的拟合我们的数据点

# 误差项定义
- 第一点： 假设a1是年龄的参数，a2 是工资的参数
- 第二点：拟合的平面：Ha(x) = a0 + a1*x1 + a2*x2(a0 是偏置项： 上下的微调)
- 第三点：整合：Ha(x) = a1*x1(所有的相加)
- 第四点：真实值和预测值之间存在着误差，用（b）表示

# 独立同分布 
- 误差：独立同分布，并且服从均值为0，方差为b2的高斯分布，（正态分布）
- 独立：张三和李四一起来贷款，他两没关系，（贷款的多少）
- 同分布： 他两都来我们这件银行
- 高斯分布：银行可能会多给，也可能会少给，绝大多数情况下，浮动不大，符合正常情况